# QKD protocols:

Change as required and create pull requests.  
Here's what we need to do:  
Basics:  
1) Ideal QKD Protocol, including verification steps.  
2) Introduce practical classical notions such as error correction and hashes, etc as required.  
3) Analysis of performance.  
  
Later on:  
1) Proper network archeticture, establishing entanglement between nodes by swapping and distillation.

Add stuff to this list and for each thing/module, open an issue. Reference the issue when you commit and make a pull request.

## Getting started  
Lets initialize required packages and modules. Make sure you have installed Python3, qiskit.

In [1]:
# Initializations
import numpy as np
import array
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit, execute, Aer
from qiskit.tools.visualization import circuit_drawer, plot_histogram

# Problem statement

Two parties, say Alice (A) and Bob (B) wish to communicate securely between themselves, while a third agent, Eve (E) wishes to evesdrop on this communication and potentially obtain information. Apart from these active agents, the communication 'channel' is suseptible to errors and loses caused by various reasons.  
The following document and codes will progress through these, in increasing order of complexity of the problem in hand.

In [2]:
class person:
    def __init__(self, name):
        self.name = name
        
class channel:
    def __init__(self, name):
        self.name = name

## Quantum Key Distribution
Classically, the most (universally) secure encryption scheme is the one time pad. This involves the two parties Ex-or a random (secret) key string to their messages before transmitting. Since the key is random, the evesdropper Eve has no information about the input without the knowledge of the key.  For this task to work though, the sender and receiver had to exchange the key at some prior point of time. This may not be secure and may lead to leaks of key due to various human factors.  
Fortunately, quantum mechanics provides a solution to this connundrum and allows Alice and Bob to exchange keys in a secure and simple manner. The following are few proposed methods and the code to simulate the same.

### BB84
Suppose Alice and Bob wish to establish a key between themselves, Alice begins with initializing two random N length bit strings $a_1[1], a_1[2] \dots a_1[N]$  and $b_1[1], b_1[2] \dots b_1[N]$ between themselves. One can locally generate purely random strings by performing measurements on qubits. She then prepares a string of qubit states $q[1], q[2] \dots q[N]$ with the following rule:  

$q[i] = |0\rangle$ when $a_1[i] = 0$ and $b_1[i] = 0$  
$q[i] = |1\rangle$ when $a_1[i] = 1$ and $b_1[i] = 0$  
$q[i] = |+\rangle$ when $a_1[i] = 0$ and $b_1[i] = 1$  
$q[i] = |-\rangle$ when $a_1[i] = 1$ and $b_1[i] = 1$  

You can notice that $a_1[]$ determines the eigenstate and $b_1[]$ determine the orientation of the same.  

In [3]:
from qiskit import qasm
class person_BB84(person):
    def __init__(self, name, N):
        super().__init__(name)
        self.length = N
        self.a = [] #key string
        self.b = [] #own basis
        self.c = [] #to store received basis
        self.d = [] #to store verification key
        self.key = []
        self.ver = []
        self.flag = 0
        self.q = QuantumCircuit(N, N)
        
    def initialize_random(self):
        ran = QuantumCircuit(1, 1)
        ran.h(0)
        ran.measure([0], [0])
        backend = Aer.get_backend('qasm_simulator')
        job_1 = execute(ran, backend, shots = self.length, memory = True)
        job_2 = execute(ran, backend, shots = self.length, memory = True)
        result_1 = job_1.result()
        result_2 = job_2.result()
        mem_1 = result_1.get_memory()
        mem_2 = result_2.get_memory()
        for x in range(self.length):
            self.a.append(int(mem_1[x],2))
            self.b.append(int(mem_2[x],2))
    
    def initialize_qubits(self): #to initialize qubits according to a, b
        for l in range(self.length):
            if self.a[l] == 1:
                self.q.x(l)
            if self.b[l] == 1:
                self.q.h(l)
                
    def measure_q(self):
        for l in range(self.length):
            if self.b[l] == 1:
                self.q.h(l)
            self.q.measure([l],[l])
        backend = Aer.get_backend('qasm_simulator')
        job_m = execute(self.q, backend, shots = 1, memory = True)
        result_m = job_m.result()
        mem = result_m.get_memory()

        self.a = []
        for l in range(self.length):
            self.a.append(int(mem[0][-l-1], 2)) # reversing order of bits and appending
        
    def compare_basis(self):
        temp = []
        for x in range(self.length):
            if self.b[x] == self.c[x]:
                temp.append(self.a[x])
        self.a = temp
        del temp
        
        self.length = len(self.a)
        
    def split_key(self, locations):
        self.key = []
        self.ver = []
        for x in range(len(locations)):
            if locations[x] == 0:
                self.key.append(self.a[x])
            else:
                self.ver.append(self.a[x])
                
    def verify_key(self):
        total = len(self.ver)
        match = 0
        for x in range(total):
            if self.ver[x] == self.d[x]:
                match+=1
        accuracy = (match*100)/total
        print("Accuracy =", accuracy)
        if accuracy > 0.95:
            print(self.name, " is good to go.")
            self.flag = 1
        else:
            print(self.name, " is facing issues.")
                    

Alice = person_BB84('Alice', 100)
Bob = person_BB84('Bob', 100)

Alice.initialize_random()
Bob.initialize_random()

Alice.initialize_qubits()

She now transmits the qubits to bob via a quantum channel. For now lets assume the channel is error free and no evesdropping.  

In [4]:
def channel_ideal(input):
    return input

Bob.q = channel_ideal(Alice.q)

Now Bob decides on a equally random bit string $b_2[1], b_2[2], \dots b_2[N]$. Bob now measures the received qubits to obtain , with basis determined by the $b_2[]$.  

If $b_2[i]$ is 0, then measure $q[i]$ in the {$|0\rangle,|1\rangle$} basis.  
If $b_2[i]$ is 1, then measure $q[i]$ in the {$|+\rangle,|-\rangle$} basis.  

In [5]:
Bob.measure_q()

After measurement, Alice and Bob publicly over classical channels exchange $b_1$ and $b_2$. They compare the strings and discard disagreeing positions (i.e)  if $b_1[i] \neq b_2[i]$ then discard $a_{1,2}[i]$.  

In [6]:
Alice.c = channel_ideal(Bob.b)
Bob.c = channel_ideal(Alice.b)

Alice.compare_basis()
Bob.compare_basis()

Now as a verification step, they select a fraction of their remaining key bits $a_{1,2}$ and communicate them publicly. If there is mismatch beyond a threshhold, then they discard the entire string and restart the process. 
If there is agreement, then they use the remaining bits as a shared secret string k.

In [7]:
def random_string_biased(length, frac):
    rand = QuantumCircuit(1, 1)
    state = [np.sqrt(1-frac) , np.sqrt(frac)]
    rand.initialize(state, [0])
    rand.measure([0], [0])
    backend = Aer.get_backend('qasm_simulator')
    job = execute(rand, backend, shots = length, memory = True)
    result = job.result()
    mem = result.get_memory()
    a = []
    for x in range(length):
        a.append(int(mem[x],2))
    return a

frac = 1/2

verify_locations = random_string_biased(len(Alice.a), frac)

Alice.split_key(verify_locations)
Bob.split_key(verify_locations)

Alice.d = channel_ideal(Bob.ver)
Bob.d = channel_ideal(Alice.ver)

Alice.verify_key()
Bob.verify_key()

if Alice.flag == 1 and Bob.flag == 1:
    print("Huston, we are good to go. I repeat we are good to go")
    print("Key: ", Alice.key)
    print("Key length: ", len(Alice.key))
else:
    print("Mission abort, mission abort")

Accuracy = 100.0
Alice  is good to go.
Accuracy = 100.0
Bob  is good to go.
Huston, we are good to go. I repeat we are good to go
Key:  [0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0]
Key length:  21
